In [1]:
import numpy as np
import pandas as pd


In [2]:
################################################################################
def arrayDist(x,y):
    return np.sqrt( (x*y).sum(axis=1) )

In [3]:
x = np.random.randint(5, size=[6,4])
y = np.random.randint(5, size=[6,4])

In [6]:
x

array([[0, 0, 0, 2],
       [3, 1, 4, 3],
       [3, 2, 3, 2],
       [3, 1, 1, 1],
       [1, 3, 1, 3],
       [3, 4, 1, 3]])

In [7]:
y

array([[4, 0, 2, 3],
       [0, 3, 4, 4],
       [4, 1, 2, 2],
       [0, 4, 1, 0],
       [4, 4, 0, 0],
       [3, 4, 1, 3]])

In [9]:
arrayDist(x[1,], y)

array([ 5.38516481,  5.56776436,  5.19615242,  2.82842712,  4.        ,
        5.09901951])

In [11]:
y*x[1,]

array([[12,  0,  8,  9],
       [ 0,  3, 16, 12],
       [12,  1,  8,  6],
       [ 0,  4,  4,  0],
       [12,  4,  0,  0],
       [ 9,  4,  4,  9]])